In [14]:
!pip install streamlit requests pandas trafilatura cloudscraper seaborn

  Using cached trafilatura-2.0.0-py3-none-any.whl.metadata (12 kB)
  Using cached cloudscraper-1.2.71-py2.py3-none-any.whl.metadata (19 kB)
ERROR: Could not find a version that satisfies the requirement localtunnel (from versions: none)
ERROR: No matching distribution found for localtunnel


In [23]:
%%writefile app.py
import streamlit as st
import pandas as pd
import requests
import time
import trafilatura
import cloudscraper
import re
import matplotlib.pyplot as plt
import seaborn as sns
import math
from datetime import datetime, timedelta
from pathlib import Path

# --- 1. 配置與 UI 設定 ---
st.set_page_config(page_title="Job Skill Analyzer", layout="wide")
st.title("🎯 Job Skill Analyzer: Data Scientist & Analyst")
st.markdown("透過 Adzuna API 抓取職缺，並深度分析技能分佈。")

# --- 2. 側邊欄設定 (已修復重複定義問題) ---
with st.sidebar:
    st.header("🔑 API Credentials")
    app_id = st.text_input("Adzuna APP ID", value="", help="請輸入您的 Adzuna Application ID")
    app_key = st.text_input("Adzuna APP KEY", type="password", value="", help="請輸入您的 Adzuna Application Key")

    st.header("🔍 Search Filters")
    country = st.selectbox("Country", ["us", "uk", "ca", "au"], index=0)
    city = st.text_input("City", value="Baltimore")
    keywords_input = st.text_input("Keywords (逗號分隔)", "data scientist, data analyst")
    keywords = [k.strip() for k in keywords_input.split(",")]

    st.header("項次設定")
    days_back = st.sidebar.slider("Days Back", 7, 90, 60)
    max_pages = st.sidebar.number_input("Max Pages", 1, 10, 3)
    results_per_page = 50

# --- 3. 技能清單 ---
SKILL_KEYWORDS = {
    'Programming': ['Python', 'SQL', ' R ', 'SAS', 'Stata', 'Julia', 'C++', 'Java', 'Scala', 'Go', 'Bash', 'Shell'],
    'Cloud & Big Data': ['AWS', 'Azure', 'GCP', 'Google Cloud', 'Snowflake', 'Databricks', 'Spark', 'Hadoop', 'Kafka', 'Redshift', 'BigQuery', 'Athena', 'Glue', 'Terraform', 'Airflow'],
    'Databases': ['PostgreSQL', 'MySQL', 'MongoDB', 'NoSQL', 'SQL Server', 'Oracle', 'Cassandra'],
    'ML & AI': ['Machine Learning', 'Deep Learning', 'Reinforcement Learning', 'NLP', 'Natural Language Processing', 'Computer Vision', 'Generative AI', 'GenAI', 'LLM', 'GPT', 'Scikit-learn', 'TensorFlow', 'PyTorch', 'Keras', 'XGBoost', 'LightGBM', 'CatBoost'],
    'Stats & Research': ['Statistics', 'Biostatistics', 'Causal Inference', 'Epidemiology', 'Econometrics', 'Bayesian', 'Survival Analysis', 'Longitudinal', 'Time Series', 'A/B Testing', 'Experimental Design', 'Propensity Score', 'Clinical Trials', 'Regression', 'Hypothesis Testing', 'RCT', 'GIS'],
    'Visualization & BI': ['Tableau', 'Power BI', 'Looker', 'Qlik', 'Matplotlib', 'Seaborn', 'Plotly', 'Shiny', 'D3.js'],
    'Engineering & DevOps': ['Git', 'GitHub', 'CI/CD', 'Docker', 'Kubernetes', 'MLOps', 'Agile', 'Scrum', 'DevOps'],
    'Degree': ['PhD', 'Ph.D.', 'Doctorate', 'Master', 'M.S.', 'MSc', 'MPH', 'MBA', 'Bachelor']
}

# --- 4. 功能函數 ---
def find_skills_refined(text):
    if pd.isna(text): return []
    res = []
    for cat, kws in SKILL_KEYWORDS.items():
        for k in kws:
            pattern = r'\b' + re.escape(k.strip()) + r'\b'
            if re.search(pattern, text, re.I):
                val = k.strip()
                if val == 'Ph.D.': val = 'PhD'
                if val == 'M.S.': val = 'Master'
                res.append(val)
    return list(set(res))

def fetch_full_description(url):
    scraper = cloudscraper.create_scraper(browser={'browser': 'chrome', 'platform': 'windows', 'desktop': True})
    try:
        response = scraper.get(url, timeout=15)
        if response.status_code == 200:
            content = trafilatura.extract(response.text)
            return content.strip() if content and len(content) > 300 else None
    except: pass
    return None

# --- 5. 主程式邏輯 ---
if st.sidebar.button("🚀 Start Fetching & Analysis"):
    if not app_id or not app_key:
        st.error("❌ 請輸入 Adzuna API ID 與 Key！")
    else:
        all_jobs = []
        cutoff_date = datetime.now() - timedelta(days=days_back)
        base_url = f"https://api.adzuna.com/v1/api/jobs/{country}/search"

        status_text = st.empty()
        progress_bar = st.progress(0)

        total_steps = len(keywords) * max_pages
        step = 0

        for kw in keywords:
            for page in range(1, max_pages + 1):
                step += 1
                status_text.info(f"📥 正在抓取: {kw} (第 {page}/{max_pages} 頁)...")
                progress_bar.progress(step / total_steps)

                params = {
                    "app_id": app_id,
                    "app_key": app_key,
                    "what": kw,
                    "where": city,
                    "results_per_page": results_per_page,
                    "sort_by": "date"
                }

                try:
                    r = requests.get(f"{base_url}/{page}", params=params)
                    r.raise_for_status()
                    data = r.json().get("results", [])
                    if not data: break

                    for job in data:
                        created_dt = datetime.fromisoformat(job.get("created").replace("Z", ""))
                        if created_dt < cutoff_date: continue

                        # 隨機抓取完整描述以獲得更精確技能 (選擇性開啟，會增加等待時間)
                        # full_desc = fetch_full_description(job.get("redirect_url"))
                        # final_desc = full_desc if full_desc else job.get("description", "")
                        final_desc = job.get("description", "") # 先用 API 提供的簡介加速測試

                        all_jobs.append({
                            "title": job.get("title"),
                            "company": job.get("company", {}).get("display_name"),
                            "created": job.get("created"),
                            "description": final_desc,
                            "url": job.get("redirect_url")
                        })
                except Exception as e:
                    st.warning(f"⚠️ API 請求中斷: {e}")
                    break
                time.sleep(0.5)

        if all_jobs:
            df = pd.DataFrame(all_jobs).drop_duplicates(subset=['url'])
            df['found_skills'] = df['description'].apply(find_skills_refined)
            df_effective = df[df['found_skills'].map(len) > 0].copy()
            n_sample = len(df_effective)

            st.success(f"✅ 完成！分析了 {len(df)} 筆職缺，其中 {n_sample} 筆包含關鍵技能。")

            # --- 數據預覽 ---
            st.subheader("📋 Latest Jobs Found")
            st.dataframe(df[['title', 'company', 'created']].head(10))

            # --- 繪製 8 個分組圖表 ---
            st.subheader("📊 Skill Distribution by Category")

            all_data = []
            for _, row in df_effective.iterrows():
                for skill in row['found_skills']:
                    for cat, ks in SKILL_KEYWORDS.items():
                        check_list = [k.strip().replace('Ph.D.', 'PhD').replace('M.S.', 'Master') for k in ks]
                        if skill in check_list:
                            all_data.append({'Category': cat, 'Skill': skill})
                            break

            full_stats = pd.DataFrame(all_data)
            categories = list(SKILL_KEYWORDS.keys())
            cols_n = 2
            rows_n = math.ceil(len(categories) / cols_n)

            fig, axes = plt.subplots(rows_n, cols_n, figsize=(16, rows_n * 5))
            axes = axes.flatten()
            palette = sns.color_palette("husl", len(categories))

            for i, cat in enumerate(categories):
                cat_df = full_stats[full_stats['Category'] == cat]
                if not cat_df.empty:
                    counts = cat_df['Skill'].value_counts().reset_index()
                    counts.columns = ['Skill', 'Count']
                    sns.barplot(data=counts, x='Count', y='Skill', ax=axes[i], color=palette[i])
                    axes[i].set_title(f'Category: {cat}', fontsize=14, fontweight='bold')
                    axes[i].set_xlim(0, n_sample + 1)

                    for p in axes[i].patches:
                        w = int(p.get_width())
                        axes[i].annotate(f'{w} ({(w/n_sample)*100:.1f}%)',
                                         (w, p.get_y() + p.get_height()/2),
                                         xytext=(5, 0), textcoords='offset points', fontsize=10)
                else:
                    axes[i].text(0.5, 0.5, 'No Data Found', ha='center', va='center', transform=axes[i].transAxes)
                    axes[i].set_title(f'Category: {cat} (Empty)', fontsize=14)

            plt.tight_layout()
            st.pyplot(fig)

            # --- 下載 ---
            csv = df.to_csv(index=False).encode('utf-8-sig')
            st.download_button("📥 Download Analysis Result (CSV)", data=csv, file_name="job_market_analysis.csv", mime="text/csv")
        else:
            st.warning("🧐 沒找到職缺，請嘗試增加 Days Back 或修改關鍵字。")

Overwriting app.py


In [24]:
!npm install localtunnel
!streamlit run app.py --server.address=localhost &>/content/logs.txt &
!npx localtunnel --port 8501 & curl https://loca.lt/mytunnelpassword

⠙⠹⠸⠼
up to date, audited 23 packages in 709ms
⠼
⠼3 packages are looking for funding
⠼  run `npm fund` for details
⠼
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠼34.48.255.141⠙your url is: https://evil-humans-send.loca.lt


In [25]:
# 1. 安裝套件
!pip install pyngrok pyngrok

In [27]:
# 2. 設定 Token (把引號內換成你的)
from pyngrok import ngrok
ngrok.set_auth_token(ngrok_api)

# Start Streamlit server on a specific port
!nohup streamlit run app.py --server.port 5011 &

# Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='5011', proto='http', bind_tls=True)

# Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://598d7cb3c5f3.ngrok-free.app
